In [1]:
import inspect
import time
from statistics import mean, stdev
from CybORG import CybORG
from CybORG.Agents import B_lineAgent, SleepAgent, GreenAgent
from CybORG.Agents.SimpleAgents.BaseAgent import BaseAgent
from CybORG.Agents.SimpleAgents.BlueReactAgent import BlueReactRemoveAgent
from CybORG.Agents.SimpleAgents.Meander import RedMeanderAgent
from CybORG.Agents.Wrappers.EnumActionWrapper import EnumActionWrapper
from CybORG.Agents.Wrappers.FixedFlatWrapper import FixedFlatWrapper
from CybORG.Agents.Wrappers.OpenAIGymWrapper import OpenAIGymWrapper
from CybORG.Agents.Wrappers.ReduceActionSpaceWrapper import ReduceActionSpaceWrapper
from CybORG.Agents.Wrappers import ChallengeWrapper
import os

from ray.tune.registry import register_env
from CybORG.Agents.Wrappers.rllib_wrapper import RLlibWrapper
import warnings
import numpy as np
from ray import air, tune

warnings.filterwarnings('ignore')

In [2]:
def env_creator(env_config: dict):
    path = str(inspect.getfile(CybORG))
    path = path[:-10] + '/Shared/Scenarios/Scenario2.yaml'
    agents = {"Red": B_lineAgent, "Green": GreenAgent}
    cyborg = CybORG(scenario_file=path, environment='sim', agents=agents)
    env = RLlibWrapper(env=cyborg, agent_name="Blue", max_steps=100)
    return env

def print_results(results_dict):
    train_iter = results_dict["training_iteration"]
    r_mean = results_dict["episode_reward_mean"]
    r_max = results_dict["episode_reward_max"]
    r_min = results_dict["episode_reward_min"]
    print(f"{train_iter:4d} \tr_mean: {r_mean:.1f} \tr_max: {r_max:.1f} \tr_min: {r_min: .1f}")

register_env(name="CybORG", env_creator=env_creator)

In [2]:
# !rm -r logs/APPO/RE3
!lshw -C display

/usr/bin/sh: 1: lshw: not found


In [3]:
from ray.rllib.algorithms.ppo import PPOConfig
from ray.rllib.policy.policy import PolicySpec

config = (
    PPOConfig()
    #Each rollout worker uses a single cpu
    .rollouts(num_rollout_workers=3, num_envs_per_worker=1, horizon=100)\
    .training(train_batch_size=3000, gamma=0.85, lr=0.00005, 
              model={"fcnet_hiddens": [256, 256], "fcnet_activation": "tanh",})\
    .environment(disable_env_checking=True, env = 'CybORG')\
    .resources(num_gpus=1)\
    .framework('tf')\
    .exploration(explore=True, exploration_config={"type": "RE3", "embeds_dim": 128, "beta_schedule": "constant", "sub_exploration": {"type": "StochasticSampling",},})\
    .offline_data(output="logs/APPO/StochasticSampling", output_compress_columns=['prev_actions', 'prev_rewards', 'dones', 't', 'eps_id', 'unroll_id', 'agent_index', 'action_prob', 'action_logp', 'action_dist_inputs', 'advantages', 'value_targets'],
                 output_config={"format": "json"},)
)
trainer = config.build()


2023-01-17 14:42:54,827	INFO worker.py:1529 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 
2023-01-17 14:43:05,217	INFO trainable.py:172 -- Trainable.setup took 12.428 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
2023-01-17 14:43:05,221	WARNING util.py:66 -- Install gputil for GPU system monitoring.


In [3]:
from tensorflow.python.client import device_lib



ModuleNotFoundError: No module named 'torch'

In [ ]:
def print_results(results_dict):
    train_iter = results_dict["training_iteration"]
    r_mean = results_dict["episode_reward_mean"]
    r_max = results_dict["episode_reward_max"]
    r_min = results_dict["episode_reward_min"]
    print(f"{train_iter:4d} \tr_mean: {r_mean:.1f} \tr_max: {r_max:.1f} \tr_min: {r_min: .1f}")

In [ ]:
for i in range(2000):
    print_results(trainer.train())

In [ ]:
from ray.rllib.algorithms.ppo import PPOConfig
from ray.rllib.policy.policy import PolicySpec

config = (
    PPOConfig()
    #Each rollout worker uses a single cpu
    .rollouts(num_rollout_workers=30, num_envs_per_worker=1, horizon=100)\
    .training(train_batch_size=3000, gamma=0.85, lr=0.00005, 
              model={"fcnet_hiddens": [256, 256], "fcnet_activation": "tanh",})\
    .environment(disable_env_checking=True, env = 'CybORG')\
    .resources(num_gpus=1)\
    .framework('tf')\
    .offline_data(output="logs/APPO/actuallystoch_otherisre3", output_compress_columns=['prev_actions', 'prev_rewards', 'dones', 't', 'eps_id', 'unroll_id', 'agent_index', 'action_prob', 'action_logp', 'action_dist_inputs', 'advantages', 'value_targets'],
                 output_config={"format": "json"},)
)
trainer = config.build()


In [ ]:
for i in range(2000):
    print_results(trainer.train())